In [ ]:
!ls

drive  sample_data


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Sequential, Linear, ReLU
from torch_geometric.nn import GINConv, global_add_pool
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score, mean_squared_error

In [ ]:
data_set = "all_test"
data_split_path = f"data/split_data/{data_set}/"
train_dc = pd.read_csv(data_split_path+"train_dc.csv")
test_dc = pd.read_csv(data_split_path+"test_dc.csv")
val_dc = pd.read_csv(data_split_path+"val_dc.csv")

def r2_rmse( g ):
    r2 =  mean_squared_error( g['synergy'], g['predict'] )
    count = len(g['synergy'])
    rmse = np.sqrt( mean_squared_error( g['synergy'], g['predict'] ) )
    return pd.Series( dict(  r2 = r2, rmse = rmse, count = count ) )

def get_top_data(r, df, top=10):
    G, P, x_1_ats, x_2_ats = r
    x_1_ats = np.array(x_1_ats)
    x_2_ats = np.array(x_2_ats)
    top = top*2
    abs_error = np.abs(G-P)
    index_top = abs_error.argsort()[:]
    values = abs_error[index_top]
    df_top = df.iloc[index_top].copy()
    df_top["log_synergy"] = G[index_top]
    df_top["predict"] = P[index_top]
    df_top["abs_error"] = values
    df_top["x_1_ats"] = x_1_ats[index_top]
    df_top["x_2_ats"] = x_2_ats[index_top]
    return df_top

In [ ]:
import numpy as np
import pandas as pd
import sys, os
from random import shuffle
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Sequential, Linear, ReLU
from torch_geometric.nn import GINConv, global_add_pool
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from utils import *
import datetime
import matplotlib.pyplot as plt
import pickle
from tqdm.notebook import tqdm

In [ ]:
un_freeze_layers = ["fc1", "fc2", "out"]

def dfs_freeze(model):
    for name, child in model.named_children():
        if(name not in un_freeze_layers):
          # print(name)
          for param in child.parameters():
              param.requires_grad = False
          dfs_freeze(child)


In [ ]:
m = nn.Sigmoid()
loss = nn.BCELoss()
input = torch.randn(3, requires_grad=True)
target = torch.empty(3).random_(2)
output = loss(m(input), target)
output.backward()

In [ ]:
def return_omics_type(data):
    t = 0
    temp = []
    name = "_"
    for k, v in data.items():
        t = t + v
        if(v):
            temp.append(k)
    temp = tuple(temp)
    if t == 1:
        return "1omics", name.join(temp)
    if t == 2:
        return "2omics", name.join(temp)
    if t == 3:
        return "3omics", name.join(temp)
        
data_types = [
    {"ge":1, "mut":1, "meth":1},
    {"ge":1, "mut":1, "meth":0},
    {"ge":1, "mut":0, "meth":1},
    {"ge":0, "mut":1, "meth":1},
    {"ge":1, "mut":0, "meth":0},
    {"ge":0, "mut":1, "meth":0},
    {"ge":0, "mut":0, "meth":1},    
]
data_sets = ["all_test"]
for data_type in data_types:
    print(data_type)
    for data_set in data_sets:
        data_path = f"data/split_data/{data_set}"
        data_processed_path = "data/split_data/{data_set}/processed/"

        """#Define model"""

        # GINConv model
        class GINConvNet(torch.nn.Module):
            def __init__(self, n_output=1,num_features_xd=78, num_features_xt=25,
                        n_filters=32, embed_dim=128, output_dim=128, dropout=0.2,
                        out_tissue_d=13, ge=0, mut=0, meth=0):

                super(GINConvNet, self).__init__()
                self.ge = ge
                self.mut = mut
                self.meth = meth
                print(self.ge, self.mut, self.meth)

                dim = 32
                self.dropout = nn.Dropout(dropout)
                self.relu = nn.ReLU()
                self.n_output = n_output
                # convolution layers
                nn1 = Sequential(Linear(num_features_xd, dim), ReLU(), Linear(dim, dim))
                self.conv1 = GINConv(nn1)
                self.bn1 = torch.nn.BatchNorm1d(dim)

                nn2 = Sequential(Linear(dim, dim), ReLU(), Linear(dim, dim))
                self.conv2 = GINConv(nn2)
                self.bn2 = torch.nn.BatchNorm1d(dim)

                nn3 = Sequential(Linear(dim, dim), ReLU(), Linear(dim, dim))
                self.conv3 = GINConv(nn3)
                self.bn3 = torch.nn.BatchNorm1d(dim)

                nn4 = Sequential(Linear(dim, dim), ReLU(), Linear(dim, dim))
                self.conv4 = GINConv(nn4)
                self.bn4 = torch.nn.BatchNorm1d(dim)

                nn5 = Sequential(Linear(dim, dim), ReLU(), Linear(dim, dim))
                self.conv5 = GINConv(nn5)
                self.bn5 = torch.nn.BatchNorm1d(dim)

                self.fc1_xd = Linear(dim, output_dim)
                
                self.drug_pt_block = Sequential(
                    Linear(3078, 1024),
                    nn.ReLU(),
                    nn.Dropout(0.3),
                    Linear(1024, 512),
                    nn.ReLU(),
                    nn.Dropout(0.3),
                    Linear(512, 128),
                    nn.ReLU(),
                    nn.Dropout(0.3)
                )
                self.drug_fc = Linear(256, 128)

                # cell line feature
                # ge 
                if self.ge:
                    self.target_ge_cnv_block = Sequential(
                        nn.Conv1d(in_channels=1, out_channels=n_filters, kernel_size=8),
                        nn.ReLU(),
                        nn.MaxPool1d(3),
                        nn.Conv1d(in_channels=n_filters, out_channels=n_filters*2, kernel_size=8),
                        nn.ReLU(),
                        nn.MaxPool1d(3),
                        nn.Conv1d(in_channels=n_filters*2, out_channels=n_filters*4, kernel_size=8),
                        nn.ReLU(),
                        nn.MaxPool1d(3),
                        nn.Flatten(),
                        nn.Linear(83584, 1024),
                        nn.Dropout(0.3),
                        nn.ReLU(),
                        nn.Linear(1024, output_dim),
                        nn.ReLU(),
                        nn.Dropout(0.3)
                        
                        
                    )
                # mut
                if self.mut:
                    self.target_mut_cnv_block = Sequential(
                        nn.Conv1d(in_channels=1, out_channels=n_filters, kernel_size=8),
                        nn.ReLU(),
                        nn.MaxPool1d(3),
                        nn.Conv1d(in_channels=n_filters, out_channels=n_filters*2, kernel_size=8),
                        nn.ReLU(),
                        nn.MaxPool1d(3),
                        nn.Conv1d(in_channels=n_filters*2, out_channels=n_filters*4, kernel_size=8),
                        nn.ReLU(),
                        nn.MaxPool1d(3),
                        nn.Flatten(),
                        nn.Linear(2944, 1024),
                        nn.ReLU(),
                        nn.Dropout(0.3),
                        nn.ReLU(),
                        nn.Linear(1024, output_dim),
                        nn.ReLU(),
                        nn.Dropout(0.3)
                        
                    )

                # meth
                if self.meth:
                    self.target_meth_cnv_block = Sequential(
                        nn.Conv1d(in_channels=1, out_channels=n_filters, kernel_size=8),
                        nn.ReLU(),
                        nn.MaxPool1d(3),
                        nn.Conv1d(in_channels=n_filters, out_channels=n_filters*2, kernel_size=8),
                        nn.ReLU(),
                        nn.MaxPool1d(3),
                        nn.Conv1d(in_channels=n_filters*2, out_channels=n_filters*4, kernel_size=8),
                        nn.ReLU(),
                        nn.MaxPool1d(3),
                        nn.Flatten(),
                        nn.Linear(1280, 1024),
                        nn.ReLU(),
                        nn.Dropout(0.3),
                        nn.ReLU(),
                        nn.Linear(1024, output_dim),
                        nn.ReLU(),
                        nn.Dropout(0.3)
                        
                    )

                # synthetic omics data
                total = self.ge + self.mut + self.meth
#                 self.synthetic_omics = nn.Linear((total)*output_dim, output_dim*max(total-1, 1))

                #attension
                self.key_xt = nn.Linear(output_dim*total, output_dim*total)
                self.key_drug = nn.Linear(output_dim, output_dim)

                self.at_fc = nn.Linear(output_dim*(total+2), 1)

                # combined layers
                self.fc1 = nn.Linear((total + 1)*output_dim, 1024)
                self.fc2 = nn.Linear(1024, output_dim)
                self.out = nn.Linear(output_dim, n_output)

                # activation and regularization
                self.relu = nn.ReLU()
                self.leaky_relu = nn.LeakyReLU()
                self.dropout = nn.Dropout(0.3)

            def forward(self, data):
        #         Batch(c_size_1=[1024], c_size_2=[1024], edge_index_1=[2, 67864], edge_index_2=[2, 68244],
        #               target=[1024, 735], x_1=[31406, 78], x_1_batch=[31406], x_2=[31559, 78], x_2_batch=[31559], y=[1024])
                x_1_batch = data.x_1_batch
                x_1, edge_index_1,  = data.x_1, data.edge_index_1
                x_pt_1 = data.xd_pt_1
                x_2_batch = data.x_2_batch
                x_2, edge_index_2,  = data.x_2, data.edge_index_2
                x_pt_2 = data.xd_pt_2

        #         drug 1
                x_1 = F.relu(self.conv1(x_1, edge_index_1))
                x_1 = self.bn1(x_1)
                x_1 = F.relu(self.conv2(x_1, edge_index_1))
                x_1 = self.bn2(x_1)
                x_1 = F.relu(self.conv3(x_1, edge_index_1))
                x_1 = self.bn3(x_1)
                x_1 = F.relu(self.conv4(x_1, edge_index_1))
                x_1 = self.bn4(x_1)
                x_1 = F.relu(self.conv5(x_1, edge_index_1))
                x_1 = self.bn5(x_1)
                x_1 = global_add_pool(x_1, x_1_batch)
                x_1 = F.relu(self.fc1_xd(x_1))
                x_1 = F.dropout(x_1, p=0.2, training=self.training)
                x_pt_1 = self.drug_pt_block(x_pt_1)
                x_1 = F.relu(self.drug_fc(torch.cat((x_1, x_pt_1), 1)))
                
        #         drug 2
                x_2 = F.relu(self.conv1(x_2, edge_index_2))
                x_2 = self.bn1(x_2)
                x_2 = F.relu(self.conv2(x_2, edge_index_2))
                x_2 = self.bn2(x_2)
                x_2 = F.relu(self.conv3(x_2, edge_index_2))
                x_2 = self.bn3(x_2)
                x_2 = F.relu(self.conv4(x_2, edge_index_2))
                x_2 = self.bn4(x_2)
                x_2 = F.relu(self.conv5(x_2, edge_index_2))
                x_2 = self.bn5(x_2)
                x_2 = global_add_pool(x_2, x_2_batch)
                x_2 = F.relu(self.fc1_xd(x_2))
                x_2 = F.dropout(x_2, p=0.2, training=self.training)
                x_pt_2 = self.drug_pt_block(x_pt_2)
                x_2 = F.relu(self.drug_fc(torch.cat((x_2, x_pt_2), 1)))


                # protein input feed-forward:
                conv_xt_ge = torch.Tensor().to(device)
                conv_xt_mut = torch.Tensor().to(device)
                conv_xt_meth = torch.Tensor().to(device)

                if self.mut:
                    target_mut = data.target_mut
                    target_mut = target_mut[:,None,:]
                    conv_xt_mut = self.target_mut_cnv_block(target_mut)

                if self.meth:
                    target_meth = data.target_meth
                    target_meth = target_meth[:,None,:]
                    conv_xt_meth = self.target_meth_cnv_block(target_meth)

                if self.ge:
                    target_ge = data.target_ge
                    target_ge = target_ge[:,None,:]
                    conv_xt_ge = self.target_ge_cnv_block(target_ge)
#                     print(conv_xt_ge.shape)
                # 1d conv layers

                xt = torch.cat((conv_xt_mut, conv_xt_meth, conv_xt_ge), 1)
#                 xt = self.synthetic_omics(xt)
#                 xt = self.relu(xt)
#                 xt = self.dropout(xt)
                

                key_xt = self.key_xt(xt)
                key_drug_1 = self.key_drug(x_1)
                key_drug_2 = self.key_drug(x_2)
#                 print(key_drug_1, key_drug_2)
                
                a_drug_1 = torch.exp(self.leaky_relu(self.at_fc(torch.cat((key_drug_1, key_xt, key_drug_2), 1))))
                a_drug_2 = torch.exp(self.leaky_relu(self.at_fc(torch.cat((key_drug_2, key_xt, key_drug_1), 1))))
#                 print(a_drug_1, a_drug_2)
                total_drug_add = a_drug_1+a_drug_2
                a_drug_1 = torch.div(a_drug_1, total_drug_add)
                a_drug_2 = torch.div(a_drug_2, total_drug_add)
                x_1 = a_drug_1*x_1
                x_2 = a_drug_2*x_2
                x_drug_combine = x_1 + x_2
#                 print(x_drug_combine)

                # concat
                xc = torch.cat((x_drug_combine, xt), 1)
                # add some dense layers
                xc = self.fc1(xc)
                xc = self.relu(xc)
                xc = self.dropout(xc)
                xc = self.fc2(xc)
                xc = self.relu(xc)
                xc = self.dropout(xc)
                out = self.out(xc)
                out = torch.sigmoid(out)

                return out, a_drug_1, a_drug_2


        # training function at each epoch
        import timeit

        def train(model, device, train_loader, optimizer, epoch, log_interval, critation):
            print('Training on {} samples...'.format(len(train_loader.dataset)))
            model.train()
        #     loss_fn = nn.MSELoss()
            avg_loss = []
            for batch_idx, data in enumerate(train_loader):
                data = data.to(device)
                optimizer.zero_grad()
                output, x_1, x_2 = model(data)
                # print("output:", output)
                # print("label:", data.y.view(-1, 1) >= 0)
                loss = critation(output, (data.y.view(-1, 1) >= 0).float().to(device))
                loss.backward()
                optimizer.step()
                avg_loss.append(loss.item())
                text = 'Train epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch,
                                                                                batch_idx * len(data.x_1),
                                                                                len(train_loader.dataset),
                                                                                100. * batch_idx / len(train_loader),
                                                                                loss.item())

        #         loop.set_description(text)
        #         loop.refresh() # to show immediately the update
            return sum(avg_loss)/len(avg_loss)

        def predicting(model, device, loader, ats=False):
            model.eval()
            total_preds = torch.Tensor()
            total_labels = torch.Tensor()
            if ats:
                x_1_predicted = torch.Tensor()
                x_2_predicted = torch.Tensor()
                print('Make prediction for {} samples...'.format(len(loader.dataset)))
                with torch.no_grad():
                    for data in loader:
                        data = data.to(device)
                        output, x_1, x_2 = model(data)
                        total_preds = torch.cat((total_preds, output.cpu()), 0)
                        total_labels = torch.cat((total_labels, data.y.view(-1, 1).cpu()), 0)
                        x_1_predicted = torch.cat((x_1_predicted, x_1.cpu()), 0)
                        x_2_predicted = torch.cat((x_2_predicted, x_2.cpu()), 0)
                return np.where(total_labels.numpy().flatten() > 0.5, 1, 0),total_preds.numpy().flatten(), x_1_predicted, x_2_predicted
            else:
                print('Make prediction for {} samples...'.format(len(loader.dataset)))
                with torch.no_grad():
                    for data in loader:
                        data = data.to(device)
                        output, x_1, x_2 = model(data)
                        total_preds = torch.cat((total_preds, output.cpu()), 0)
                        total_labels = torch.cat((total_labels, data.y.view(-1, 1).cpu()), 0)
                return np.where(total_labels.numpy().flatten() > 0.5, 1, 0),np.where(total_preds.numpy().flatten() > 0.5, 1, 0)

        def draw(list_, label, y_label, title):
            plt.figure()
            plt.plot(list_, label=label)
            plt.title(title)
            plt.xlabel('Epoch')
            plt.ylabel(y_label)
            plt.legend()
            # save image
            plt.savefig(title+".png")  # should before show method

        """#Load data"""

        model_st = "GINConvNet"
        dataset = 'GDSC'
        train_batch = 1024
        val_batch = 1024
        test_batch = 1024
        log_interval = 20

        print('\nrunning on ', model_st + '_' + dataset )
        train_data = TestbedDataset(root=data_path, dataset=dataset+"_"+'train_dc')
        val_data = TestbedDataset(root=data_path, dataset=dataset+"_"+'val_dc')
        test_data = TestbedDataset(root=data_path, dataset=dataset+"_"+'test_dc')

        # val_mix_dc = TestbedDataset(root=data_path, dataset=dataset+"_"+'mix_val')
        # val_blind_cell = TestbedDataset(root=data_path, dataset=dataset+"_"+'blind_cell_val')
        # val_blind_1_drug = TestbedDataset(root=data_path, dataset=dataset+"_"+'blind_1_drug_val')
        # val_blind_1_drug_cell = TestbedDataset(root=data_path, dataset=dataset+"_"+'blind_1_drug_cell_val')
        # val_blind_2_drug = TestbedDataset(root=data_path, dataset=dataset+"_"+'blind_2_drug_val')
        # val_blind_all = TestbedDataset(root=data_path, dataset=dataset+"_"+'blind_all_val')


        # test_mix_dc = TestbedDataset(root=data_path, dataset=dataset+"_"+'mix_test')
        # test_blind_cell = TestbedDataset(root=data_path, dataset=dataset+"_"+'blind_cell_test')
        # test_blind_1_drug = TestbedDataset(root=data_path, dataset=dataset+"_"+'blind_1_drug_test')
        # test_blind_1_drug_cell = TestbedDataset(root=data_path, dataset=dataset+"_"+'blind_1_drug_cell_test')
        # test_blind_2_drug = TestbedDataset(root=data_path, dataset=dataset+"_"+'blind_2_drug_test')
        # test_blind_all = TestbedDataset(root=data_path, dataset=dataset+"_"+'blind_all_test')

        # make data PyTorch
        # mini-batch processing ready
        train_loader = DataLoader(train_data, batch_size=train_batch, shuffle=True, follow_batch=['x_1', 'x_2'])
        val_loader = DataLoader(val_data, batch_size=val_batch, shuffle=False, follow_batch=['x_1', 'x_2'])
        test_loader = DataLoader(test_data, batch_size=test_batch, shuffle=False, follow_batch=['x_1', 'x_2'])

        # val_mix_dc_loader = DataLoader(val_mix_dc, batch_size=test_batch, shuffle=False, follow_batch=['x_1', 'x_2'])
        # val_blind_cell_loader = DataLoader(val_blind_cell, batch_size=test_batch, shuffle=False, follow_batch=['x_1', 'x_2'])
        # val_blind_1_drug_loader = DataLoader(val_blind_1_drug, batch_size=test_batch, shuffle=False, follow_batch=['x_1', 'x_2'])
        # val_blind_1_drug_cell_loader = DataLoader(val_blind_1_drug_cell, batch_size=test_batch, shuffle=False, follow_batch=['x_1', 'x_2'])
        # val_blind_2_drug_loader = DataLoader(val_blind_2_drug, batch_size=test_batch, shuffle=False, follow_batch=['x_1', 'x_2'])
        # val_blind_all_loader = DataLoader(val_blind_all, batch_size=test_batch, shuffle=False, follow_batch=['x_1', 'x_2'])

        # test_mix_dc_loader = DataLoader(test_mix_dc, batch_size=test_batch, shuffle=False, follow_batch=['x_1', 'x_2'])
        # test_blind_cell_loader = DataLoader(test_blind_cell, batch_size=test_batch, shuffle=False, follow_batch=['x_1', 'x_2'])
        # test_blind_1_drug_loader = DataLoader(test_blind_1_drug, batch_size=test_batch, shuffle=False, follow_batch=['x_1', 'x_2'])
        # test_blind_1_drug_cell_loader = DataLoader(test_blind_1_drug_cell, batch_size=test_batch, shuffle=False, follow_batch=['x_1', 'x_2'])
        # test_blind_2_drug_loader = DataLoader(test_blind_2_drug, batch_size=test_batch, shuffle=False, follow_batch=['x_1', 'x_2'])
        # test_blind_all_loader = DataLoader(test_blind_all, batch_size=test_batch, shuffle=False, follow_batch=['x_1', 'x_2'])

        print("CPU/GPU: ", torch.cuda.is_available())

        """#Training

        ##Init model
        """

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(device)
        modeling = GINConvNet(**data_type)
        model = modeling.to(device)

        """##Load linear model and """

        """##Define paramters"""

        lr = 0.001
        num_epoch = 200
        best_ret_test = None
        print('Learning rate: ', lr)
        print('Epochs: ', num_epoch)

        train_losses = []
        val_losses = []
        val_pearsons = []

        val_mix_dc_losses = []
        val_blind_cell_losses = []
        val_blind_1_drug_losses = []
        val_blind_1_drug_cell_losses = []
        val_blind_2_drug_losses = []
        val_blind_all_losses = []

        val_mix_dc_pearsons = []
        val_blind_cell_pearsons = []
        val_blind_1_drug_pearsons = []
        val_blind_1_drug_cell_pearsons = []
        val_blind_2_drug_pearsons = []
        val_blind_all_pearsons = []

        """##Train model"""
        omics, name_omics = return_omics_type(data_type)
        # regression_model_path = "model/save_model/" + f"GIN_ADD_AT_CLS/{omics}/{name_omics}/{data_set}" + "/"
        save_path = "model/save_model/" + f"ALL_TEST_GIN_ADD_AT_CLS/{omics}/{name_omics}/{data_set}" + "/"
        print(save_path)
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        best_mse = 1000
        best_pearson = 1
        best_epoch = -1

        best_val_losses = 10000
        best_val_mix_dc_losses = 10000
        best_val_blind_cell_losses = 10000
        best_val_blind_1_drug_losses = 10000
        best_val_blind_1_drug_cell_losses = 10000
        best_val_blind_2_drug_losses = 10000
        best_val_blind_all_losses = 10000


        ret_test_save = [1,1]
        ret_test_mix_dc_save = [1,1] 
        ret_test_blind_cell_save = [1,1]
        ret_test_blind_1_drug_save = [1,1]
        ret_test_blind_1_drug_cell_save = [1,1]
        ret_test_blind_2_drug_save = [1,1]
        ret_test_blind_all_save = [1,1]


        model_file_name = save_path + 'best_model' + '.model'
        current_file_model = save_path + 'current_model_' + '.model'
        model_all_file_name = save_path + 'model_all' + '.model'
        model_mix_file_name = save_path + 'model_mix' + '.model'
        model_blind_cell_file_name = save_path + 'model_blind_cell' + '.model'
        model_blind_1_drug_file_name = save_path + 'model_blind_1_drug' + '.model'
        model_blind_1_drug_cell_file_name = save_path + 'model_blind_1_drug_cell' + '.model'
        model_blind_2_drug_file_name = save_path + 'model_blind_2_drug' + '.model'
        model_blind_all_file_name = save_path + 'model_blind_all' + '.model'

        result_file_name = save_path + 'result_' + model_st + '_' + dataset +  '.csv'

        loss_fig_name = save_path + 'model_' + model_st + '_' + dataset + '_loss'
        pearson_fig_name = save_path + 'model_' + model_st + '_' + dataset + '_pearson'

        loss_fig_name_mix_dc = save_path + 'model_' + model_st + '_' + dataset + '_loss_mix_dc'
        pearson_fig_name_mix_dc = save_path + 'model_' + model_st + '_' + dataset + '_pearson_mix_dc'

        loss_fig_name_blind_cell = save_path + 'model_' + model_st + '_' + dataset + '_loss_blind_cell'
        pearson_fig_name_blind_cell = save_path + 'model_' + model_st + '_' + dataset + '_pearson_blind_cell'

        loss_fig_name_blind_1_drug = save_path + 'model_' + model_st + '_' + dataset + '_loss_blind_1_drug'
        pearson_fig_name_blind_1_drug = save_path + 'model_' + model_st + '_' + dataset + '_pearson_blind_1_drug'

        loss_fig_name_blind_1_drug_cell = save_path + 'model_' + model_st + '_' + dataset + '_loss_blind_1_drug_cell'
        pearson_fig_name_blind_1_drug_cell = save_path + 'model_' + model_st + '_' + dataset + '_pearson_blind_1_drug_cell'

        loss_fig_name_blind_2_drug = save_path + 'model_' + model_st + '_' + dataset + '_loss_blind_2_drug'
        pearson_fig_name_blind_2_drug = save_path + 'model_' + model_st + '_' + dataset + '_pearson_blind_2_drug'

        loss_fig_name_blind_all = save_path + 'model_' + model_st + '_' + dataset + '_loss_blind_all'
        pearson_fig_name_blind_all = save_path + 'model_' + model_st + '_' + dataset + '_pearson_blind_all'

        val_mix_dc_losses = []
        val_blind_cell_losses = []
        val_blind_1_drug_losses = []
        val_blind_1_drug_cell_losses = []
        val_blind_2_drug_losses = []
        val_blind_all_losses = []

        val_mix_dc_pearsons = []
        val_blind_cell_pearsons = []
        val_blind_1_drug_pearsons = []
        val_blind_1_drug_cell_pearsons = []
        val_blind_2_drug_pearsons = []
        val_blind_all_pearsons = []

        if os.path.exists(model_file_name):
          model.load_state_dict(torch.load(model_file_name, map_location=torch.device('cpu')))
          dfs_freeze(model)
          print("load and freeze model")

        """
        #Continue Training"""

        # #load model
        # if os.path.exists(current_file_model):
        #   model.load_state_dict(torch.load(current_file_model))
        #   with open(save_path+ "log.pickle", "rb") as f:
        #     log = pickle.load(f)
        # #   train_losses, val_losses,\
        # #   val_mix_dc_losses, val_blind_cell_losses,\
        # #   val_blind_1_drug_losses, val_blind_1_drug_cell_losses,\
        # #   val_blind_2_drug_losses, val_blind_all_losses,\
        # #   val_mix_dc_pearsons, val_blind_cell_pearsons,\
        # #   val_blind_1_drug_pearsons, val_blind_1_drug_cell_pearsons,\
        # #   val_blind_2_drug_pearsons, val_blind_all_pearsons\
        # #   = log
        #   best_mse = min(val_losses)
        #   best_pearson = 1
        #   best_epoch = -1
        #   print("load previous model")

        def return_ret(G, P):
            return [bce(G,P),bce(G,P)]

        from IPython.display import clear_output 
        # loss_fn = nn.MSELoss()
        loss_fn = nn.BCELoss()
        for epoch in range(num_epoch):
            print('Epoch {} | Training on {} samples...'.format(epoch, len(train_loader.dataset)))
            avg_loss = []    
            train_loss = train(model, device, train_loader, optimizer, epoch+1, log_interval, loss_fn)
            #VAL:
            G,P = predicting(model, device, val_loader)
        #     G_mix_dc, P_mix_dc = predicting(model, device, val_mix_dc_loader)
        #     G_blind_cell, P_blind_cell = predicting(model, device, val_blind_cell_loader)
        #     G_blind_1_drug, P_blind_1_drug = predicting(model, device, val_blind_1_drug_loader)
        #     G_blind_1_drug_cell, P_blind_1_drug_cell = predicting(model, device, val_blind_1_drug_cell_loader)
        #     G_blind_2_drug, P_blind_2_drug = predicting(model, device, val_blind_2_drug_loader)
        #     G_blind_all, P_blind_all = predicting(model, device, val_blind_all_loader)


            ret = return_ret(G, P)
        #     ret_mix_dc = return_ret(G_mix_dc, P_mix_dc)
        #     ret_blind_cell = return_ret(G_blind_cell, P_blind_cell)
        #     ret_blind_1_drug = return_ret(G_blind_1_drug, P_blind_1_drug)
        #     ret_blind_1_drug_cell = return_ret(G_blind_1_drug_cell, P_blind_1_drug_cell)
        #     ret_blind_2_drug = return_ret(G_blind_2_drug, P_blind_2_drug)
        #     ret_blind_all = return_ret(G_blind_all, P_blind_all)




            train_losses.append(train_loss)
            val_losses.append(ret[1])
            # val_pearsons.append(ret[2])

        #     val_mix_dc_losses.append(ret_mix_dc[1])
        #     val_blind_cell_losses.append(ret_blind_cell[1])
        #     val_blind_1_drug_losses.append(ret_blind_1_drug[1])
        #     val_blind_1_drug_cell_losses.append(ret_blind_1_drug_cell[1])
        #     val_blind_2_drug_losses.append(ret_blind_2_drug[1])
        #     val_blind_all_losses.append(ret_blind_all[1])


        #     val_mix_dc_pearsons.append(ret_mix_dc[2])
        #     val_blind_cell_pearsons.append(ret_blind_cell[2])
        #     val_blind_1_drug_pearsons.append(ret_blind_1_drug[2])
        #     val_blind_1_drug_cell_pearsons.append(ret_blind_1_drug_cell[2])
        #     val_blind_2_drug_pearsons.append(ret_blind_2_drug[2])
        #     val_blind_all_pearsons.append(ret_blind_all[2])

            if ret[1]<best_val_losses:
                best_val_losses = ret[1]
                G_test,P_test = predicting(model, device, test_loader)
                ret_test_save = return_ret(G_test, P_test)
                print("RMSE all test improved")
                torch.save(model.state_dict(), model_file_name)

                result_dict = {
                    "test": (predicting(model, device, test_loader, ats=True), test_dc),
                #     "mix": (G_mix_dc_test, P_mix_dc_test, mix_test),
                #     "blind_cell": (G_blind_cell_test, P_blind_cell_test, blind_cell_test),
                #     "blind_1_drug": (G_blind_1_drug_test, P_blind_1_drug_test, blind_1_drug_test),
                #     "blind_1_drug_cell": (G_blind_1_drug_cell_test, P_blind_1_drug_cell_test, blind_1_drug_cell_test),
                #     "blind_2_drug": (G_blind_2_drug_test, P_blind_2_drug_test, blind_2_drug_test),
                #     "blind_all": (G_blind_all_test, P_blind_all_test, blind_all_test)   
                }


                for key, value in tqdm(result_dict.items()):
                    temp = get_top_data(value[0], value[1])
                #     top_2_drug = temp.groupby(['Drug1', 'Drug2']).apply( r2_rmse ).reset_index().sort_values(by=['rmse'])
                #     top_1_drug = temp.groupby(['Drug1']).apply( r2_rmse ).reset_index().sort_values(by=['rmse'])
                #     top_2_drug.to_csv(save_path+"save_top/"+key+"2_drug"+".csv", index=False)
                #     top_1_drug.to_csv(save_path+"save_top/"+key+"1_drug"+".csv", index=False)
                temp.to_csv(save_path+"_cls_detail_result.csv", index=False)
        #     if ret_mix_dc[1]<best_val_mix_dc_losses:
        #         best_val_mix_dc_losses = ret_mix_dc[1]
        #         G_mix_dc_test, P_mix_dc_test = predicting(model, device, test_mix_dc_loader)
        #         ret_test_mix_dc_save = return_ret(G_mix_dc_test, P_mix_dc_test)
        #         print("RMSE mix test improved")
        #     if ret_blind_cell[1]<best_val_blind_cell_losses:
        #         best_val_blind_cell_losses = ret_blind_cell[1]
        #         G_blind_cell_test, P_blind_cell_test = predicting(model, device, test_blind_cell_loader)
        #         ret_test_blind_cell_save = return_ret(G_blind_cell_test, P_blind_cell_test)
        #         print("RMSE blind cell test improved")
        #     if ret_blind_1_drug[1]<best_val_blind_1_drug_losses:
        #         best_val_blind_1_drug_losses = ret_blind_1_drug[1]
        #         G_blind_1_drug_test, P_blind_1_drug_test = predicting(model, device, test_blind_1_drug_loader)
        #         ret_test_blind_1_drug_save = return_ret(G_blind_1_drug_test, P_blind_1_drug_test)
        #         print("RMSE blind 1 drug improved")
        #     if ret_blind_1_drug_cell[1]<best_val_blind_1_drug_cell_losses:
        #         best_val_blind_1_drug_cell_losses = ret_blind_1_drug_cell[1]
        #         G_blind_1_drug_cell_test, P_blind_1_drug_cell_test = predicting(model, device, test_blind_1_drug_cell_loader)
        #         ret_test_blind_1_drug_cell_save = return_ret(G_blind_1_drug_cell_test, P_blind_1_drug_cell_test)
        #         print("RMSE blind 1 drug cell improved")
        #     if ret_blind_2_drug[1]<best_val_blind_2_drug_losses:
        #         best_val_blind_2_drug_losses = ret_blind_2_drug[1]
        #         G_blind_2_drug_test, P_blind_2_drug_test = predicting(model, device, test_blind_2_drug_loader)
        #         ret_test_blind_2_drug_save = return_ret(G_blind_2_drug_test, P_blind_2_drug_test)
        #         print("RMSE blind 2 drug improved")
        #     if ret_blind_all[1]<best_val_blind_all_losses:
        #         best_val_blind_all_losses = ret_blind_all[1]
        #         G_blind_all_test, P_blind_all_test = predicting(model, device, test_blind_all_loader)
        #         ret_test_blind_all_save = return_ret(G_blind_all_test, P_blind_all_test)
        #         print("RMSE blind all improved")


            with open(result_file_name,'w') as f:
                f.write('ret_test: '+','.join(map(str,ret_test_save)) +"\n")
        #         f.write('ret_mix_dc: '+','.join(map(str,ret_test_mix_dc_save)) +"\n")
        #         f.write('ret_blind_cell: '+','.join(map(str,ret_test_blind_cell_save)) +"\n")
        #         f.write('ret_blind_1_drug: '+','.join(map(str,ret_test_blind_1_drug_save)) +"\n")
        #         f.write('ret_blind_1_drug_cell: '+','.join(map(str,ret_test_blind_1_drug_cell_save)) +"\n")
        #         f.write('ret_blind_2_drug: '+','.join(map(str,ret_test_blind_2_drug_save)) +"\n")
        #         f.write('ret_blind_all: '+','.join(map(str,ret_test_blind_all_save)) +"\n")

            draw_loss(train_losses, val_losses, loss_fig_name)
            # draw_pearson(val_pearsons, pearson_fig_name)

        #     draw_loss(train_losses, val_mix_dc_losses, loss_fig_name_mix_dc)
        #     draw_pearson(val_mix_dc_pearsons, pearson_fig_name_mix_dc)

        #     draw_loss(train_losses, val_blind_cell_losses, loss_fig_name_blind_cell)
        #     draw_pearson(val_blind_cell_pearsons, pearson_fig_name_blind_cell)

        #     draw_loss(train_losses, val_blind_1_drug_losses, loss_fig_name_blind_1_drug)
        #     draw_pearson(val_blind_1_drug_pearsons, pearson_fig_name_blind_1_drug)

        #     draw_loss(train_losses, val_blind_1_drug_cell_losses, loss_fig_name_blind_1_drug_cell)
        #     draw_pearson(val_blind_1_drug_cell_pearsons, pearson_fig_name_blind_1_drug_cell)

        #     draw_loss(train_losses, val_blind_2_drug_losses, loss_fig_name_blind_2_drug)
        #     draw_pearson(val_blind_2_drug_pearsons, pearson_fig_name_blind_2_drug)

        #     draw_loss(train_losses, val_blind_all_losses, loss_fig_name_blind_all)
        #     draw_pearson(val_blind_all_pearsons, pearson_fig_name_blind_all)

            torch.save(model.state_dict(), current_file_model)
            log = [
                train_losses, val_losses,\
        #           val_mix_dc_losses, val_blind_cell_losses,\
        #           val_blind_1_drug_losses, val_blind_1_drug_cell_losses,\
        #           val_blind_2_drug_losses, val_blind_all_losses,\
        #           val_mix_dc_pearsons, val_blind_cell_pearsons,\
        #           val_blind_1_drug_pearsons, val_blind_1_drug_cell_pearsons,\
        #           val_blind_2_drug_pearsons, val_blind_all_pearsons
                ]

            with open(save_path+ "/log.pickle", "wb") as f:
                pickle.dump(log, f)

        # data_split_path = f"data/split_data/{data_set}/"
        # train_dc = pd.read_csv(data_split_path+"train_dc.csv")
        # test_dc = pd.read_csv(data_split_path+"test_dc.csv")
        # val_dc = pd.read_csv(data_split_path+"val_dc.csv")

        # mix_val = pd.read_csv(data_split_path+"mix_val.csv")
        # mix_test = pd.read_csv(data_split_path+"mix_test.csv")

        # blind_cell_val = pd.read_csv(data_split_path+"blind_cell_val.csv")
        # blind_cell_test = pd.read_csv(data_split_path+"blind_cell_test.csv")

        # blind_1_drug_val = pd.read_csv(data_split_path+"blind_1_drug_val.csv")
        # blind_1_drug_test = pd.read_csv(data_split_path+"blind_1_drug_test.csv")

        # blind_1_drug_cell_val = pd.read_csv(data_split_path+"blind_1_drug_cell_val.csv")
        # blind_1_drug_cell_test = pd.read_csv(data_split_path+"blind_1_drug_cell_test.csv")

        # blind_2_drug_val = pd.read_csv(data_split_path+"blind_2_drug_val.csv")
        # blind_2_drug_test = pd.read_csv(data_split_path+"blind_2_drug_test.csv")

        # blind_all_val = pd.read_csv(data_split_path+"blind_all_val.csv")
        # blind_all_test = pd.read_csv(data_split_path+"blind_all_test.csv")

          # result_dict = {
          #     "test": (predicting(model, device, test_loader, ats=True), test_dc),
          # #     "mix": (G_mix_dc_test, P_mix_dc_test, mix_test),
          # #     "blind_cell": (G_blind_cell_test, P_blind_cell_test, blind_cell_test),
          # #     "blind_1_drug": (G_blind_1_drug_test, P_blind_1_drug_test, blind_1_drug_test),
          # #     "blind_1_drug_cell": (G_blind_1_drug_cell_test, P_blind_1_drug_cell_test, blind_1_drug_cell_test),
          # #     "blind_2_drug": (G_blind_2_drug_test, P_blind_2_drug_test, blind_2_drug_test),
          # #     "blind_all": (G_blind_all_test, P_blind_all_test, blind_all_test)   
          # }


          # for key, value in tqdm(result_dict.items()):
          #     temp = get_top_data(value[0], value[1])
          # #     top_2_drug = temp.groupby(['Drug1', 'Drug2']).apply( r2_rmse ).reset_index().sort_values(by=['rmse'])
          # #     top_1_drug = temp.groupby(['Drug1']).apply( r2_rmse ).reset_index().sort_values(by=['rmse'])
          # #     top_2_drug.to_csv(save_path+"save_top/"+key+"2_drug"+".csv", index=False)
          # #     top_1_drug.to_csv(save_path+"save_top/"+key+"1_drug"+".csv", index=False)
          # temp.to_csv(save_path+"_cls_detail_result.csv", index=False)
        
        import gc
        del model
        del train_data 
        del val_data 
        del test_data 
        torch.cuda.empty_cache() 
        gc.collect()

{'ge': 1, 'mut': 1, 'meth': 1}

running on  GINConvNet_GDSC
Pre-processed data found: data/split_data/all_test/processed/GDSC_train_dc.pt, loading ...
Pre-processed data found: data/split_data/all_test/processed/GDSC_val_dc.pt, loading ...
Pre-processed data found: data/split_data/all_test/processed/GDSC_test_dc.pt, loading ...
CPU/GPU:  False
cpu
1 1 1
Learning rate:  0.001
Epochs:  200
model/save_model/ALL_TEST_GIN_ADD_AT_CLS/3omics/ge_mut_meth/all_test/
Epoch 0 | Training on 11512 samples...
Training on 11512 samples...
Make prediction for 8964 samples...
Make prediction for 8968 samples...
RMSE all test improved
Make prediction for 8968 samples...


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1 | Training on 11512 samples...
Training on 11512 samples...
Make prediction for 8964 samples...
Epoch 2 | Training on 11512 samples...
Training on 11512 samples...
Make prediction for 8964 samples...
Epoch 3 | Training on 11512 samples...
Training on 11512 samples...
Make prediction for 8964 samples...
Make prediction for 8968 samples...
RMSE all test improved
Make prediction for 8968 samples...


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 4 | Training on 11512 samples...
Training on 11512 samples...
Make prediction for 8964 samples...
Make prediction for 8968 samples...
RMSE all test improved
Make prediction for 8968 samples...


...

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 55 | Training on 11512 samples...
Training on 11512 samples...
Make prediction for 8964 samples...
Epoch 56 | Training on 11512 samples...
Training on 11512 samples...
Make prediction for 8964 samples...
Epoch 57 | Training on 11512 samples...
Training on 11512 samples...
Make prediction for 8964 samples...
Epoch 58 | Training on 11512 samples...
Training on 11512 samples...
Make prediction for 8964 samples...
Epoch 59 | Training on 11512 samples...
Training on 11512 samples...
Make prediction for 8964 samples...
Epoch 60 | Training on 11512 samples...
Training on 11512 samples...
Make prediction for 8964 samples...
Epoch 61 | Training on 11512 samples...
Training on 11512 samples...
Make prediction for 8964 samples...
Epoch 62 | Training on 11512 samples...
Training on 11512 samples...
Make prediction for 8964 samples...
Epoch 63 | Training on 11512 samples...
Training on 11512 samples...
Make prediction for 8964 samples...
Epoch 64 | Training on 11512 samples...
Training on 115

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-bc48740e137f>", line 649, in <module>
    with open(result_file_name,'w') as f:
OSError: [Errno 107] Transport endpoint is not connected: 'model/save_model/ALL_TEST_GIN_ADD_AT_CLS/3omics/ge_mut_meth/all_test/result_GINConvNet_GDSC.csv'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 1823, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'OSError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/ultratb.py", line 1132, in get_records
    return _fixed_

OSError: ignored